In [ ]:
import pandas as pd
import plotly.express as px
import data_preprocessing.data_preprocess as dp
import numpy as np
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy.spatial.distance import cdist
from scipy.stats import multivariate_normal
from scipy.spatial.distance import cdist


In [2]:
# Interesting variables to be considered for policy space
policy_vars = [
    "Military: Positive",
    "European Community/Union: Positive",
    "Freedom and Human Rights",
    "Democracy",
    "Political Corruption",
    "Environmental Protection",
    "Welfare State",
    "Right-left position",
    "Planned Economy",
    "Equality: Positive"]

In [3]:
party_scaled, voter_scaled = dp.get_scaled_party_voter_data(x_var='Democracy', y_var='Education Expansion')

c:\PERSONAL\TUM\Case Studies\Case_Study_TUM25-8\data_preprocessing\data_loading.py:40: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)
c:\PERSONAL\TUM\Case Studies\Case_Study_TUM25-8\data_preprocessing\data_loading.py:57: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_filtered = df_filtered.apply(pd.to_numeric, errors="ignore")


In [4]:
party_scaled

,Country,Date,Calendar_Week,Party_Name,Democracy,Education Expansion,Democracy Scaled,Education Expansion Scaled,Democracy Voters_Mean,Education Expansion Voters_Mean,Democracy Combined,Education Expansion Combined,Label
0,Germany,26/09/2021,202109,90/Greens,5.544,2.987,1.306876,0.253244,-0.380168,0.062598,-0.042759,0.100728,90/Greens
1,Germany,26/09/2021,202109,LINKE,5.414,3.047,1.175452,0.328147,0.222400,0.025477,0.413010,0.086011,LINKE
2,Germany,26/09/2021,202109,SPD,3.864,3.210,-0.391529,0.531634,-0.227540,-0.043127,-0.260337,0.071825,SPD
3,Germany,26/09/2021,202109,FDP,4.785,4.253,0.539561,1.833699,-0.036908,-0.015348,0.078386,0.354461,FDP
4,Germany,26/09/2021,202109,CDU/CSU,2.559,1.802,-1.710825,-1.226092,-0.297121,-0.016302,-0.579861,-0.258260,CDU/CSU
5,Germany,26/09/2021,202109,AfD,3.558,1.842,-0.700881,-1.176157,1.076211,0.028612,0.720793,-0.212342,AfD


In [5]:
voter_scaled

,Democracy,Education Expansion,who did you vote for:second vote(a),"do you incline towards a party, if so which one(a)",how strongly do you incline towards this party,Party_Name,party_choice,Democracy Scaled,Education Expansion Scaled,Label
0,2.75,4.0,6.0,6.0,2.0,90/Greens,0,-0.137181,0.442277,Voter
1,3.25,4.0,4.0,4.0,1.0,SPD,1,0.567327,0.442277,Voter
2,2.00,3.5,1.0,1.0,1.0,CDU/CSU,2,-1.193943,-0.080815,Voter
3,3.00,4.0,6.0,6.0,2.0,90/Greens,0,0.215073,0.442277,Voter
4,2.75,3.5,322.0,808.0,0.0,AfD,3,-0.137181,-0.080815,Voter
...,...,...,...,...,...,...,...,...,...,...
6869,3.75,4.5,4.0,5.0,3.0,SPD,1,1.271835,0.965369,Voter
6870,2.25,4.0,1.0,1.0,2.0,CDU/CSU,2,-0.841689,0.442277,Voter
6871,1.50,2.0,4.0,1.0,2.0,SPD,1,-1.898451,-1.650092,Voter
6872,3.00,4.0,4.0,808.0,0.0,SPD,1,0.215073,0.442277,Voter


In [154]:
# Plot without scaling
fig = px.scatter(
    pd.concat([
        voter_scaled.assign(Type="Voter", Size=5, Color="Voter"),
        party_scaled.assign(Type="Party", Size=15, Color=party_scaled["Party_Name"])
    ]),
    x="Democracy",
    y="Education Expansion",
    color="Color",
    symbol="Type",
    size="Size",
    title="Unscaled Voter and Party Positions"
)
fig.update_traces(textposition="top center")
fig.show()

In [155]:
concatenated_df = pd.concat([voter_scaled, party_scaled], ignore_index=True)

fig = px.scatter(
    concatenated_df,
    x='Democracy Scaled',
    y='Education Expansion Scaled',
    color='Label',
    symbol='Label')
fig.update_traces(marker=dict(size=10))
fig.update_layout(title='Scaled Voter and Party Positions')
fig.show()

In [60]:
from scipy.stats import gaussian_kde
import numpy as np

x_var = "Democracy"
y_var = "Education Expansion"

# Your voter cloud (already scaled to [0,10])
x = voter_scaled[f"{x_var} Scaled"].values
y = voter_scaled[f"{y_var} Scaled"].values

# Stack for 2D
data = np.vstack([x, y])

# Build KDE — you now have a continuous density function!
kde = gaussian_kde(data, bw_method='scott')  # or 'silverman', or custom float

# Evaluate anywhere in 2D space:
density_at_5_5 = kde([5, 5])  # returns a scalar density estimate


In [61]:
x1=voter_scaled[f"{x_var} Scaled"].values
x.__len__()

6874

In [23]:
density_at_5_5

array([8.04306789e-70])

In [13]:
def voter_density(x_input, y_input):
    """
    Continuous density function over 2D space from KDE.
    
    Parameters:
    -----------
    x_input, y_input : float or np.ndarray
        Coordinates in policy space (can be scalars or arrays).
    
    Returns:
    --------
    float or np.ndarray
        Density estimate(s) at the given location(s).
    """
    xy = np.vstack([np.ravel(x_input), np.ravel(y_input)])
    density_vals = kde(xy)
    return density_vals.reshape(np.shape(x_input))


In [29]:
# Scalar evaluation
voter_density(5, 5)   # → float

# Grid evaluation (e.g. for plotting or integration)
X, Y = np.meshgrid(np.linspace(-5, 5, 100), np.linspace(-5, 5, 100))
Z = voter_density(X, Y)  # shape (100, 100)

In [25]:
voter_density(5,5)

array(8.04306789e-70)

In [30]:
from sklearn.mixture import GaussianMixture
import numpy as np

# Step 1: Prepare your 2D data
X = voter_scaled[[f"{x_var} Scaled", f"{y_var} Scaled"]].values

# Step 2: Fit GMM — choose number of components (e.g. 4)
gmm = GaussianMixture(n_components=4, covariance_type='full', random_state=0)
gmm.fit(X)


GaussianMixture(n_components=4, random_state=0)

In [16]:
from scipy.stats import multivariate_normal

def gmm_density(x_input, y_input):
    """
    Evaluate the GMM density at given (x, y).
    
    Parameters:
    -----------
    x_input, y_input : float or np.ndarray
        Coordinates in policy space.
    
    Returns:
    --------
    float or np.ndarray
        Density value(s).
    """
    x_flat = np.ravel(x_input)
    y_flat = np.ravel(y_input)
    points = np.column_stack([x_flat, y_flat])
    
    density_vals = np.zeros(len(points))
    for weight, mean, cov in zip(gmm.weights_, gmm.means_, gmm.covariances_):
        rv = multivariate_normal(mean=mean, cov=cov)
        density_vals += weight * rv.pdf(points)
    
    return density_vals.reshape(np.shape(x_input))


In [42]:
Xgrid, Ygrid = np.meshgrid(np.linspace(-5, 5, 100), np.linspace(-5, 5, 100))
Z = gmm_density(Xgrid, Ygrid)


In [41]:
gmm_density(5,5)

array(9.41235279e-21)

In [33]:
print("Weights:", gmm.weights_)
print("Means:\n", gmm.means_)
print("Covariances:\n", gmm.covariances_)


Weights: [0.23312948 0.38313235 0.13000181 0.25373636]
Means:
 [[ 0.48400193 -0.33105269]
 [-0.50499498  0.4758824 ]
 [ 1.20562554  1.03488626]
 [-0.2998731  -0.94462177]]
Covariances:
 [[[ 0.79791284 -0.03497077]
  [-0.03497077  0.29927697]]

 [[ 0.52483532 -0.13149103]
  [-0.13149103  0.49950782]]

 [[ 0.61067214 -0.00910017]
  [-0.00910017  0.25792051]]

 [[ 0.66768692 -0.17979253]
  [-0.17979253  0.89606824]]]


In [19]:
from scipy.stats import multivariate_normal
import numpy as np

def gmm_indefinite_integral(x, y):
    total_cdf = 0
    point = np.array([x, y])
    for w, mu, cov in zip(gmm.weights_, gmm.means_, gmm.covariances_):
        total_cdf += w * multivariate_normal.cdf(point, mean=mu, cov=cov)
    return total_cdf


In [201]:
def gmm_density_and_loggrad(x_input, y_input, gmm):
    """
    Evaluate the GMM density and gradient of log-density at given (x, y).

    Parameters:
    -----------
    x_input, y_input : float or np.ndarray
        Coordinates in policy space.
    gmm : fitted sklearn.mixture.GaussianMixture object

    Returns:
    --------
    density_vals : np.ndarray
        GMM density values at input points.
    grad_log_density : np.ndarray
        Gradient of log-density at input points. Shape: (N, 2)
    """
    x_flat = np.ravel(x_input)
    y_flat = np.ravel(y_input)
    points = np.column_stack([x_flat, y_flat])  # Shape: (N, 2)
    N = len(points)

    density_vals = np.zeros(N)
    grad = np.zeros_like(points)

    # Loop over components
    for weight, mean, cov in zip(gmm.weights_, gmm.means_, gmm.covariances_):
        rv = multivariate_normal(mean=mean, cov=cov)
        pdf_vals = rv.pdf(points)                          # (N,)
        diff = points - mean                               # (N, 2)
        inv_cov = np.linalg.inv(cov)                       # (2, 2)
        grad_comp = -pdf_vals[:, None] * (diff @ inv_cov.T)  # (N, 2)

        density_vals += weight * pdf_vals
        grad += weight * grad_comp

    # Compute ∇ log p(x) = ∇p(x) / p(x)
    eps = 1e-9  # to avoid divide-by-zero
    grad_log_density = grad / (density_vals[:, None] + eps)

    # Reshape density to match input shape
    density_vals = density_vals.reshape(np.shape(x_input))

    return grad_log_density


In [206]:
F_x = gmm_density_and_loggrad(X_t[0,:],X_t[1,:],gmm)
F_x

array([[-3.70227076e-13, -5.66928749e-13],
       [-9.27461994e-06, -2.35783006e-06],
       [ 1.90457664e-05,  3.85396124e-06],
       ...,
       [ 8.33036269e-04,  4.30604146e-04],
       [-1.41979254e-50, -2.17648997e-50],
       [-2.38587917e-10,  3.78025067e-11]], shape=(6874, 2))

In [212]:
history = [data.copy()]
history[-1]

array([[-0.13718124,  0.56732666, -1.19394307, ..., -1.89845096,
         0.21507271,  1.62408849],
       [ 0.44227702,  0.44227702, -0.08081524, ..., -1.65009203,
         0.44227702, -0.60390751]], shape=(2, 6874))

In [ ]:
def run_simulation(data,T,N,D,sigma_noise,gmm_components,alpha,beta,gamma):
    history = [data.copy()]
    
    for t in range(T):
    # Current positions
        X_t = history[-1]

    # Compute weight matrix w_ij = exp(-||x_i - x_j||)
        distances = cdist(X_t.T, X_t.T,metric='euclidean')  
        W = np.exp(-distances)
        W /= W.sum(axis=1, keepdims=True)

    # Weighted average term
        weighted_sum = W @ X_t.T
        #print(weighted_sum.shape)
        
    # Drift term: -∇ log gmm_density(x_i)
        F_x=gmm_density_and_loggrad(X_t[0,:],X_t[1,:],gmm)
       

    # Error term (Gaussian noise)
        noise = np.random.normal(0, sigma_noise, size=(N, D))
        
    # Update rule
        X_next = alpha*weighted_sum*X_t.T - beta*F_x + gamma*noise

        history.append(X_next.T)
        
    final_positions = history[-1]
    return final_positions    

In [216]:
N = x.__len__()             
D = 2                
T = 500               
sigma_noise = 0.05   
gmm_components = 4 

sim=run_simulation(data,10,N,D,sigma_noise,gmm_components,1,1,1)

(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)


In [ ]:
def plot_with_simulation_separate(concatenated_df, simulation_points):
    fig = px.scatter(
        concatenated_df,
        x='Democracy Scaled',
        y='Education Expansion Scaled',
        color='Label',
        symbol='Label'
    )
    
    # Add simulation points as a separate trace
    fig.add_scatter(
        x=simulation_points[0],
        y=simulation_points[1],
        mode='markers',
        marker=dict(
            color='rgba(0,0,0,0.2)',
            size=4,
            symbol='circle'
        ),
        name='Simulation Points'
    )
    
    fig.update_layout(title='Scaled Positions with Simulation Overlay')
    return fig

In [229]:
sim=run_simulation(data,100,N,D,sigma_noise,gmm_components,0.1,0.1,0.1)
plot_with_simulation_separate(concatenated_df,sim)

In [218]:
sim=run_simulation(data,100,N,D,sigma_noise,gmm_components,1,1,1)
plot_with_simulation_separate(concatenated_df,sim)

(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)
(6874, 2)


In [220]:
sim=run_simulation(data,100,N,D,sigma_noise,gmm_components,1,0.1,1)
plot_with_simulation_separate(concatenated_df,sim)

In [221]:
sim=run_simulation(data,100,N,D,sigma_noise,gmm_components,0.5,0.5,1)
plot_with_simulation_separate(concatenated_df,sim)

In [222]:
sim=run_simulation(data,100,N,D,sigma_noise,gmm_components,0.1,1,0.5)
plot_with_simulation_separate(concatenated_df,sim)

In [223]:
sim=run_simulation(data,100,N,D,sigma_noise,gmm_components,1,0.1,1)
plot_with_simulation_separate(concatenated_df,sim)

In [ ]:
sim